# 1. Download SP500 Price Data

## 1.1 Import all necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplt
import statsmodels.api as sm
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from statsmodels.regression.rolling import RollingOLS

In [3]:
warnings.filterwarnings('ignore')

## 1.2 Download the SP500 Constituent Data

In [6]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
print(sp500)

[    Symbol            Security  ...      CIK      Founded
0      MMM                  3M  ...    66740         1902
1      AOS         A. O. Smith  ...    91142         1916
2      ABT              Abbott  ...     1800         1888
3     ABBV              AbbVie  ...  1551152  2013 (1888)
4      ACN           Accenture  ...  1467373         1989
..     ...                 ...  ...      ...          ...
498    XYL          Xylem Inc.  ...  1524472         2011
499    YUM         Yum! Brands  ...  1041061         1997
500   ZBRA  Zebra Technologies  ...   877212         1969
501    ZBH       Zimmer Biomet  ...  1136869         1927
502    ZTS              Zoetis  ...  1555280         1952

[503 rows x 8 columns],                   Date  ...                                             Reason
                  Date  ...                                             Reason
0        April 3, 2024  ...                   Market capitalization change.[4]
1        April 3, 2024  ...              

## 1.3 Clean the Data

- Isolate the ticker symbols

In [ ]:
tickers = 

# 2. Calculate Different Technical Indicators per Stock

# 3. Aggregate on Monthley Level and Filter per Month the Most Liquid Stocks

# 4. Calculate Monthly Returns for Different Time-Horizons

# 5. Download Fama-French Factors; Calculate Rolling Factor Betas Per Stock

# 6.  Per Month, Create a K-means Clustering Model to group similar assets based on their Features

# 7. Per Month, Select Assets Based on the Cluster and Build a Portfolio Based on That

# 8. Visualize the Portfolio Returns and Compare That with the SP500 